In [2]:
from pyfluids import Fluid, Input, FluidsList
import numpy as np, csv

n=40
trunc = lambda x: round(x,min(-int(np.floor(np.log10(abs(x))))+4,0))
air = Fluid(FluidsList.Air)
P = np.array([trunc(10**(x/n)) for x in range(2*n,int(6.5*n)+1)]).transpose()
T = np.arange(150, 1500, 5, dtype=int)
R_ideal = 8.3144598
props = ["VOLUME","CONDUCTIVITY","Cp","G", "GAMMAs", "H", "M", "PRANDTL_NUMBER", "RHO", "S", "SON_VEL", "U", "VISC"]
tabs = dict()
for p in props:
    tabs[p] = np.zeros((T.shape[0]+1, P.shape[0]+1))
    tabs[p][0,1:] = P
    tabs[p][1:,0] = T.transpose()
for i in range(1,P.shape[0]+1):
    for j in range(1,T.shape[0]+1):
        Pin = Input.pressure(P[i-1])
        Tin = Input.temperature(T[j-1]-273.15)
        s = air.with_state(Pin, Tin)
        tabs["RHO"][j,i] = s.density
        tabs["H"][j,i] = s.enthalpy
        tabs["S"][j,i] = s.entropy
        tabs["Cp"][j,i] = s.specific_heat
        tabs["VOLUME"][j,i] = 1/s.density
        tabs["VISC"][j,i] = s.dynamic_viscosity
        tabs["U"][j,i] = s.internal_energy
        tabs["PRANDTL_NUMBER"][j,i] = s.prandtl
        tabs["M"][j,i] = s.molar_mass
        tabs["GAMMAs"][j,i] = 1 + R_ideal/(s.specific_heat*s.molar_mass - R_ideal)
        tabs["CONDUCTIVITY"][j,i] = s.conductivity
        tabs["SON_VEL"][j,i] = s.sound_speed
#print(tabs["M"])

In [3]:
for prop, tab in tabs.items():
    with open(f"air/{prop}.csv", mode="w") as f:
        writer = csv.writer(f, lineterminator="\n")
        writer.writerows(tab)
s